# MLflow Training Tutorial

This example predicts the quality of wine using [sklearn.linear_model.ElasticNet](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html).  

> This example is based on the `examples/sklearn_elasticnet_wine/train.ipynb` jupyter notebook

Attribution
* The data set used in this example is from http://archive.ics.uci.edu/ml/datasets/Wine+Quality
* P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
* Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.


## Training

In [ ]:
# Wine Quality Sample
def train(in_alpha, in_l1_ratio):
    import os
    import warnings
    import sys
    
    import pandas as pd
    import numpy as np
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import ElasticNet
    
    import mlflow
    import mlflow.sklearn
    
    import logging
    logging.basicConfig(level=logging.WARN)
    logger = logging.getLogger(__name__)
    
    def eval_metrics(actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Read the wine-quality csv file
    csv_path = 'examples/sklearn_elasticnet_wine/winequality-red.csv'
    data = pd.read_csv(csv_path, sep=';')
    
    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)
    
    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    
    # Set default values if no alpha is provided
    if float(in_alpha) is None:
        alpha = 0.5
    else:
        alpha = float(in_alpha)
    
    # Set default values if no l1_ratio is provided
    if float(in_l1_ratio) is None:
        l1_ratio = 0.5
    else:
        l1_ratio = float(in_l1_ratio)
    
    # Useful for multiple runs (only doing one run in this example)    
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)
        
        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)
        
        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        
        mlflow.sklearn.log_model(lr, "model")

In [ ]:
train(0.5, 0.5)

In [ ]:
train(0.2, 0.2)

In [ ]:
train(0.1, 0.1)

## Visualizing Experiment Runs

**Execute in a Jupyter terminal**

```sh
mlflow ui --host 0.0.0.0
```
Visit http://localhost:5000


## Packing the Model

**Execute in a Jupyter Terminal**

* Activate the conda environement (include in the docker image):

```sh
conda activate sklearn_example
```

* Test the MLFlow project

```sh
cd /ws
mlflow run --env-manager=local examples/sklearn_elasticnet_wine
```

## Serving the Model

* Deploy model as web service

```sh
RUN_ID=f4b2aa55b3224494a339177da4961aee     # REPLACE WITH YOUR RUN ID
mlflow models serve  --env-manager=local -m file:///ws/mlruns/0/${RUN_ID}/artifacts/model -p 1234
```

* Predict wine quality via web service

```json
{
    "columns": [
        "alcohol", 
        "chlorides", 
        "citric acid", 
        "density", 
        "fixed acidity", 
        "free sulfur dioxide", 
        "pH", 
        "residual sugar", 
        "sulphates", 
        "total sulfur dioxide", 
        "volatile acidity"
    ],
    "data":[
        [12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]
    ]
}
```

In [ ]:
!curl  \
    -X \
    POST http://127.0.0.1:1234/invocations  \
    -H "Content-Type:application/json; format=pandas-split" \
    --data '{"columns":["alcohol", "chlorides", "citric acid", "density", "fixed acidity", "free sulfur dioxide", "pH", "residual sugar", "sulphates", "total sulfur dioxide", "volatile acidity"],"data":[[12.8, 0.029, 0.48, 0.98, 6.2, 29, 3.33, 1.2, 0.39, 75, 0.66]]}'